# АВТОРИЗАЦИЯ В API
Перед тем как познакомиться с API ВКонтакте необходимо пройти авторизацию, т. к. многие методы требуют сервисного токена. Также мы познакомимся с форматом файлов YAML, который позволяет удобно работать с параметрами скрипта и паролями.

Авторизация применяется практически во всех API, чтобы отдавать данные только их владельцу или контролировать количество запросов в единицу времени. Сервисный токен для нашей задачи создается вместе с новым приложением. Приложение мы делать, конечно, не будем. Оно нужно только для получения токена, чтобы сделать необходимые выгрузки.

Зайдите на страницу https://vk.com/editapp?act=create, чтобы создать приложение (вы должны быть авторизованы ВКонтакте). Дайте приложению любое название и оставьте значение платформы "Standalone-приложение".

После подтверждения создания приложения в приложении ВК или по СМС зайдите в настройки:



Нужный нам токен лежит в поле "Сервисный ключ доступа".

# ФОРМАТ YAML
Теперь необходимо передавать этот ключ с каждым запросом с методом wall.get. Возникает проблема: как хранить этот ключ. Просто записать его в переменную в коде не лучшая идея.

Это относится вообще к любым токенам, паролям, ключам и другой чувствительной информации по крайней мере по двум причинам:

При отправке кому-либо файла с кодом или сохранении в другом месте ваш ключ может быть доступен другому человеку. Еще хуже, если вы случайно сохраните его на общем сетевом диске или открытом репозитории на Github.
Ключ, который продолжительное время виден на вашем экране (пока вы пишете код), также не добавляет безопасности вашему приложению.
Для частичного решения этих проблем, а также хранения внешних переменных вне кода, был разработан формат YAML. 

Он позволяет записывать ваши переменные и их значения в файл, а затем импортировать их в код в JSON-формате. Это гораздо удобнее, чем читать обычный файл построчно и доставать из него нужные вам строки. Также в YAML-файлах можно оставлять комментарии к параметрам.

Т. е. его отличие YAML-формата от обычного текстового файла в том, что Python (и другие языки программирования) может распознавать его содержимое как словарь.

In [12]:
from yaml import load, dump
from bs4 import BeautifulSoup
import requests

In [8]:
f = open('./module11_files/config_example.yaml', 'r', encoding='utf-8')
config = load(f)
f.close()

In [3]:
print(config)

{'access_token': 'g6434uqghoq374gh3qh38ry24orh24h3rjg03q', 'mysql': {'host': '10.0.108.8', 'port': 3306, 'database': 'scoring', 'user': 'defaultuser', 'password': 'passw@rd'}, 'cities': ['Москва', 'Санкт-Петербург', 'Волгоград', 'Новороссийск', 'Тула', 'Мурманск', 'Смоленск', 'Севастополь', 'Одесса', 'Киев', 'Керчь', 'Минск']}


В нем для примера содержится пример ключ-значение для хранения единичного значения. Для импорта токена с код достаточно написать:

In [4]:
token = config['access_token']
token

'g6434uqghoq374gh3qh38ry24orh24h3rjg03q'

Второй пример - хранение конфигураций подключения к базе данных в виде словаря. Параметры импортируются аналогично:

In [6]:
database_host = config['mysql']['host']
database_host

'10.0.108.8'

Третий пример - хранение списка значений в виде обычного листа:

In [7]:
print(config['cities'])

['Москва', 'Санкт-Петербург', 'Волгоград', 'Новороссийск', 'Тула', 'Мурманск', 'Смоленск', 'Севастополь', 'Одесса', 'Киев', 'Керчь', 'Минск']


# СОЗДАЙТЕ КЛЮЧ ДОСТУПА
Создайте файл config.yaml и запишите в него сервисный токен вашего приложения ВКонтакте. Используйте ключ 'access_token' как в примере, далее в коде мы будем использовать такое обозначение.

In [11]:
with open('./module11_files/config.yaml', 'w', encoding='utf-8') as f:
    data = {'access_token': '8e6bc4568e6bc4568e6bc4567d8e0c9be188e6b8e6bc456d262bc5ce618161076adb05e'}
    dump(data, f)

# ОСНОВНЫЕ ПОНЯТИЯ API


В прошлом блоке мы рассматривали получение данных сайтов из HTML-страниц, что во многих случаях довольно неудобно. К счастью, многие системы имеют специальный интерфейс для забора данных программным способом с помощью API.

API (application programming interface) - набор методов и параметров, которые позволяют отдавать структурированную информацию по запросу.

В этом блоке мы изучим работу API ВКонтакте на открытых данных – это один из самых простых API.

В итоге мы научимся получать данные для статистических отчетов произвольной группы, например:

соотношение мужчин и женщин в группе;
статистика географии пользователей;
другие данные для аналитики групп конкурентов.
Но сначала рассмотрим работу API на простом примере, на основе которого работают многие системы. 

Перейдите по следующей ссылке в браузере, подставив сервисный токен из прошлого шага:
https://api.vk.com/method/users.get?user_id=1&v=5.52&access_token=token

In [14]:
with open('./module11_files/config.yaml', 'r', encoding='utf-8') as f:
    token = load(f)['access_token']
url = 'https://api.vk.com/method/users.get?user_id=1&v=5.52&access_token={}'.format(token)
r = requests.get(url)

In [15]:
r.json()

{'response': [{'id': 1, 'first_name': 'Pavel', 'last_name': 'Durov'}]}

Сейчас мы сделали GET-запрос к API ВКонтакте, который состоит из следующих частей:

https://api.vk.com/method - домен и URL запроса API. Обычно не меняется
users.get - название метода, который отдает определенный отчет. В нашем случае это метод для получения информации о пользователе
user_id и v - параметры запроса: идентификатор пользователя, о котором хотим получить информацию, и номер версии API
Данный метод не требует авторизации в системе, поскольку эти данные являются открытыми.

Однако, очень часто запросы к API требуют авторизационного токена, о них мы поговорим позже. Токен выдается только тем пользователям, которые имеют право просматривать определенные данные. Например, показания счетчиков Яндекс.Метрики вашего проекта.

На все остальные запросы без корректного токена система отвечает отказом.

Если мы посмотрим документацию метода users.get, то в ней описаны множество других параметров, которые можно получить о пользователе: дата рождения, пол, родной город и другие.

Т. е. все то, что мы видим на странице пользователя в интерфейсе или приложении ВКонтакте (конечно, если пользователь их указал). Добавим к запросу дату рождения и пол (согласно документации эти параметры надо перечислять в поле fields):
https://api.vk.com/method/users.get?user_id=1&v=5.52&fields=sex,bdate&access_token=token

In [17]:
url = 'https://api.vk.com/method/users.get?user_id=1&v=5.52&fields=sex,bdate,country&access_token={}'.format(token)
requests.get(url).json()

{'response': [{'id': 1,
   'first_name': 'Pavel',
   'last_name': 'Durov',
   'sex': 2,
   'bdate': '10.10.1984',
   'country': {'id': 1, 'title': 'Russia'}}]}

# Запросы к API из кода
Мы делали все запросы в браузере. Давайте реализуем аналогичные запросы в коде.
Теперь помимо URL запроса (https://api.vk.com...) нам надо указывать его параметры (user_id, fields и другие). Создадим для этого словарь params:

In [21]:
url = 'https://api.vk.com/method/users.get'
params = {
    'user_id': 1,
    'v': 5.52,
    'fields': 'sex,bdate,country',
    'access_token': token
}

In [27]:
r = requests.get(url, params=params)

Мы получили ответ в формате JSON. Этот формат представляет из себя набор вложенных словарей и листов, к которым очень удобно обращаться при обработке ответа. JSON применяется в подавляющем большинстве систем с API, т. к. намного проще и удобнее, чем его предшественник - формат XML.

Для улучшения читаемости можно воспользоваться библиотекой pprint. Она сильно помогает читать JSON-формат для больших ответов.

In [28]:
from pprint import pprint

data = r.json()

pprint(data)

{'response': [{'bdate': '10.10.1984',
               'country': {'id': 1, 'title': 'Russia'},
               'first_name': 'Pavel',
               'id': 1,
               'last_name': 'Durov',
               'sex': 2}]}


Получим из этого запроса дату рождения. Для этого надо сначала обратиться к ключу response:

In [30]:
data['response'][0]['bdate']

'10.10.1984'

# Упражнение
(1 возможный балл)
Имеется набор ID пользователей users. Необходимо посчитать какую долю этих пользователей составляют женщины. Не учитывайте пользователей, у которых пол не указан. Пример формата ответа: 0.35.

users = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Ниже в качестве подсказки приведены первые шаги в решении задачи. Но сначала попробуйте справиться сами, так интереснее!

In [33]:
import pandas as pd
from pandas.io.json import json_normalize

In [41]:
url = 'https://api.vk.com/method/users.get'
params = {
#     'user_id': 1,
    'v': 5.52,
    'fields': 'sex,bdate,country',
    'access_token': token
}

users = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
df = pd.DataFrame()

for user in users:
    params['user_id'] = user
    data = requests.get(url, params=params).json()
    df = pd.concat([df, json_normalize(data['response'])])

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  from ipykernel import kernelapp as app


In [63]:
df.reset_index()

,index,bdate,country.id,country.title,deactivated,first_name,id,last_name,sex
0,0,10.10.1984,1.0,Russia,NaN,Pavel,1,Durov,2
1,0,NaN,NaN,NaN,NaN,Alexandra,2,Vladimirova,1
2,0,NaN,NaN,NaN,deleted,DELETED,3,,0
3,0,NaN,NaN,NaN,deleted,DELETED,4,,0
4,0,18.11,1.0,Russia,NaN,Ilya,5,Perekopsky,2
5,0,NaN,1.0,Russia,NaN,Nikolay,6,Durov,2
6,0,NaN,49.0,United Kingdom,NaN,Alexey,7,Kobylyansky,2
7,0,NaN,2.0,Ukraine,NaN,Aki,8,Sepiashvili,2
8,0,NaN,1.0,Russia,NaN,Nastya,9,Vasilyeva,1
9,0,NaN,1.0,Russia,NaN,Alexander,10,Kuznetsov,2


In [61]:
df[df['sex']==1]['sex'].count() / df[df['sex'] != 0]['sex'].count()

0.25

# НАЧАЛО РЕШЕНИЯ
Начнем с цикла запросов к API ВКонтакте:
```python
for user in users:

    params = {

        'user_id': user,

        'v': 5.52,

        'fields': 'sex,bdate',

        'access_token': token

    }

    

    r = requests.get(url, params = params)

    print(r.json())
```
Результат:

{'response': [{'id': 1, 'first_name': 'Павел', 'last_name': 'Дуров', 'sex': 2, 'bdate': '10.10.1984'}]}

{'response': [{'id': 2, 'first_name': 'Александра', 'last_name': 'Владимирова', 'sex': 1, 'hidden': 1}]}

{'response': [{'id': 3, 'first_name': 'DELETED', 'last_name': '', 'deactivated': 'deleted', 'sex': 0}]}

{'response': [{'id': 4, 'first_name': 'DELETED', 'last_name': '', 'deactivated': 'deleted', 'sex': 0}]}

{'response': [{'id': 5, 'first_name': 'Илья', 'last_name': 'Перекопский', 'sex': 2, 'bdate': '18.11'}]}

{'response': [{'id': 6, 'first_name': 'Николай', 'last_name': 'Дуров', 'sex': 2, 'hidden': 1}]}

{'response': [{'id': 7, 'first_name': 'Алексей', 'last_name': 'Кобылянский', 'sex': 2, 'hidden': 1}]}

{'response': [{'id': 8, 'first_name': 'Аки', 'last_name': 'Сепиашвили', 'sex': 2}]}

{'response': [{'id': 9, 'first_name': 'Настя', 'last_name': 'Васильева', 'sex': 1}]}

{'response': [{'id': 10, 'first_name': 'Александр', 'last_name': 'Кузнецов', 'sex': 2}]}

Вам необходимо дописать алгоритм, чтобы посчитать долю пользователей с sex = 1 среди всех пользователей со значением sex 1 или 2.